In [20]:
import org.apache.spark.sql.types._

In [2]:
val df = (spark.read
  .option("header", true)
  .option("inferSchema", true)
  .json("data/people.json"))
  
df.groupBy("age").count()

[age: bigint, count: bigint]

In [3]:
val df = (spark.read
  .option("header", true)
  .option("inferSchema", true)
  .json("data/people.json"))

# Reading data from Implied Schema

In [26]:
val explicitSchema = StructType(
  StructField("name", StringType, false) ::
  StructField("city", StringType, true) ::
  StructField("country", StringType, true) ::
  StructField("age", IntegerType, true) :: Nil
)

val inExplicit = (spark.readStream
  .schema(explicitSchema)
  .format("csv")
  .option("header", true)
  .option("maxFilesPerTrigger", 1)
  .load("data/people.csv"))

In [27]:
val inImplied = (spark.read
  .format("csv")
  .option("header", true)
  .option("inferSchema", true)
  .load("data/people.csv"))
  
val impliedSchema = inImplied.schema

println(impliedSchema)
println(explicitSchema)

StructType(StructField(Name,StringType,true), StructField( City,StringType,true), StructField(Country,StringType,true), StructField(Age,IntegerType,true))
StructType(StructField(name,StringType,false), StructField(city,StringType,true), StructField(country,StringType,true), StructField(age,IntegerType,true))


# Reading data using Spark Streaming

In [28]:
import org.apache.spark._
import org.apache.spark.streaming._
val ssc = new StreamingContext(sc, Seconds(1))
val lines = ssc.socketTextStream("localhost", 9999)
val words = lines.flatMap(_.split("\\W+"))
val wordCounts = words.map(word => (word, 1)).reduceByKey(_ + _)

wordCounts.print()

ssc.start()
ssc.awaitTerminationOrTimeout(10)